<a href="https://colab.research.google.com/github/nicks165/VectorDatabases/blob/main/Pinecone_evaluation_cohere_wiki_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cohere example

In [2]:
!pip install -U cohere pinecone-client datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
from datasets import load_dataset
import torch
import cohere




#Load all documents + embeddings
limit = -1 # keep -1 for all, else update to a positive number to limit

max_docs_loaded = 0
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-zh-embeddings", split="train", streaming=True)

docs = []
doc_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    doc_embeddings.append(doc['emb'])
    max_docs_loaded += 1
    if (limit > 0 and max_docs_loaded == limit):
      break

In [7]:
import numpy as np

shape = np.array(doc_embeddings).shape
print(shape)

(1, 768)


In [8]:
import pinecone

pinecone.init(api_key="199c7b9a-c651-4fb0-b5b9-4bc3f706b9b3",
              environment="us-east-1-aws")


In [12]:
index_name = 'cohere-wiki-2m'
dimension = shape[1]
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=dimension,
        metric='cosine'
    )

# now connect to the index
index = pinecone.Index(index_name)

In [1]:
import time
batch_size = 100

ids = [str(i) for i in range(shape[0])]

# create list of metadata dictionaries
meta = [{'text': text['text']} for text in docs]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, doc_embeddings, meta))

start_time = time.time()
for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

    if(i_end in [shape[0], batch_size, 1000, 10000, 100000, 500000, 1000000, 2000000]):
      print(" For {0} entries, time taken for inserts = {1} ".format(i_end, time.time() - start_time))


# let's view the index statistics
print("==========================================")
print(index.describe_index_stats())

In [19]:
co = cohere.Client(f"o7lTEJeC1QHjU5I4Ee6U2I0m6l5wCOUPWqwoGM7H")  # Add your cohere API key from www.cohere.com

query = "What was the cause of the major recession in the early 20th century?"

# create the query embedding
xq = co.embed(texts=[query], model='multilingual-22-12').embeddings

query_start_time = time.time()

# query, returning the top 5 most similar results
res = index.query(xq, top_k=5, include_metadata=True)

print(" For 1 query, time taken for search = {0} ".format(time.time() - query_start_time))

for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

 For 1 query, time taken for search = 0.7168543338775635 
0.75: 习近平（），陝西富平人，祖籍河南邓州，中国共产党和中华人民共和国政治人物，2012年当选中共中央总书记和中共中央军委主席，成为最高领导人。2013年当选国家主席和国家军委主席。中华人民共和国第五代最高领导人。
